In [1]:
import json

import pandas as pd

In [2]:
hotels = pd.read_csv('hotels.csv')
hotels.head(3)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643


In [3]:
hotels.isna().sum().sort_values(ascending=False)

lng                                           2448
lat                                           2448
positive_review                                  0
days_since_review                                0
tags                                             0
reviewer_score                                   0
total_number_of_reviews_reviewer_has_given       0
review_total_positive_word_counts                0
hotel_address                                    0
additional_number_of_scoring                     0
review_total_negative_word_counts                0
negative_review                                  0
reviewer_nationality                             0
hotel_name                                       0
average_score                                    0
review_date                                      0
total_number_of_reviews                          0
dtype: int64

In [4]:
with open("lng_dict.json", "r") as infile:
    lng_dict = json.load(infile)

In [5]:
with open("lat_dict.json", "r") as infile:
    lat_dict = json.load(infile)

In [6]:
missings_idx = hotels[hotels.lng.isna()].index
hotels.loc[missings_idx, 'lat'] = hotels.loc[missings_idx, 'hotel_address'].map(lat_dict)
hotels.loc[missings_idx, 'lng'] = hotels.loc[missings_idx, 'hotel_address'].map(lng_dict)

In [7]:
hotels.isna().sum().sort_values(ascending=False)

hotel_address                                 0
positive_review                               0
lat                                           0
days_since_review                             0
tags                                          0
reviewer_score                                0
total_number_of_reviews_reviewer_has_given    0
review_total_positive_word_counts             0
total_number_of_reviews                       0
additional_number_of_scoring                  0
review_total_negative_word_counts             0
negative_review                               0
reviewer_nationality                          0
hotel_name                                    0
average_score                                 0
review_date                                   0
lng                                           0
dtype: int64

In [12]:
hotels = hotels.select_dtypes([], ['object'])

In [13]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [14]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [15]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [16]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  

# Создаём модель  
regr = RandomForestRegressor(n_estimators=100, n_jobs=-1)  

# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  

# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [17]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.14132619369015134
